# Section 3: Model Training（モデル学習）

## 🎯 このセクションの目標
- Snowpark MLでの機械学習モデル構築方法を学ぶ
- XGBoostを使ってチャーン予測モデルを学習する
- **クロスバリデーション** で汎化性能を確認する
- **Feature Importance** で重要な特徴量を把握する
- **SHAP値** でモデルの解釈可能性を確保する

## 📋 予測タスク
**2024年6月30日時点のデータで、2024年後半に顧客がチャーン（離反）するかどうかを予測する2値分類**


---
## 3.1 環境設定

> 💡 **パッケージについて**: `environment.yml` により必要なパッケージ（snowflake-ml-python, shap, matplotlib等）は自動的にインストールされています。


In [ ]:
# Warningを抑制（ハンズオン向け）
import warnings
warnings.filterwarnings('ignore')

from snowflake.snowpark.context import get_active_session
from snowflake.snowpark import functions as F
from snowflake.snowpark.types import *

# 可視化・解釈ライブラリ
import shap
import matplotlib.pyplot as plt
import numpy as np

# セッションの取得
session = get_active_session()

# 環境設定
session.use_database("MLOPS_HOL_DB")
session.use_schema("FEATURE_STORE")
session.use_warehouse("MLOPS_HOL_SQL_WH")

print("✅ ライブラリのインポート完了")


---
## 3.2 学習データの準備


In [ ]:
# Section 2で作成した学習データセットを読み込み
training_data = session.table("TRAINING_DATASET_V1")

print(f"学習データのレコード数: {training_data.count():,}")
print(f"カラム数: {len(training_data.columns)}")

training_data.show(5)


In [ ]:
# 特徴量カラムとラベルカラムの定義
FEATURE_COLS = [
    # RFM特徴量（Recency, Frequency, Monetary）
    "DAYS_SINCE_LAST_ORDER",   # Recency: 最終注文からの日数
    "TOTAL_ORDER_COUNT",        # Frequency: 総注文回数
    "ORDER_COUNT_2024_H1",      # Frequency: 2024年前半の注文回数
    "TOTAL_ORDER_AMOUNT",       # Monetary: 総注文金額
    "AVG_ORDER_AMOUNT",         # Monetary: 平均注文金額
    "TOTAL_AMOUNT_2024_H1",     # Monetary: 2024年前半の注文金額
    # 行動特徴量
    "RETURN_RATE"               # 返品率
]

LABEL_COL = "IS_CHURNED"
ID_COL = "CUSTOMER_ID"

print(f"特徴量数: {len(FEATURE_COLS)}")
print(f"ラベル: {LABEL_COL}")


In [ ]:
# 必要なカラムのみ選択
model_data = training_data.select(
    [ID_COL] + FEATURE_COLS + [LABEL_COL]
)


In [ ]:
# ラベルの分布確認（クラス不均衡のチェック）
print("=== ラベル分布 ===")
label_dist = model_data.group_by(LABEL_COL).count().collect()
for row in label_dist:
    label = "チャーン" if row[LABEL_COL] == 1 else "アクティブ"
    print(f"  {label} ({row[LABEL_COL]}): {row['COUNT']:,} 件")

total = sum([row['COUNT'] for row in label_dist])
churn_rate = [row['COUNT'] for row in label_dist if row[LABEL_COL] == 1][0] / total * 100
print(f"\nチャーン率: {churn_rate:.1f}%")


---
## 3.3 訓練・テストデータの分割


In [ ]:
# データの分割（80% 訓練、20% テスト）
train_df, test_df = model_data.random_split(
    weights=[0.8, 0.2],
    seed=42
)

print(f"訓練データ: {train_df.count():,} 件")
print(f"テストデータ: {test_df.count():,} 件")


In [ ]:
# 訓練・テストデータのラベル分布確認
print("=== 訓練データのラベル分布 ===")
train_df.group_by(LABEL_COL).count().show()

print("=== テストデータのラベル分布 ===")
test_df.group_by(LABEL_COL).count().show()


---
## 3.4 ハイパーパラメータチューニング

**ハイパーパラメータチューニング** は、モデルの性能を最大化するために、
最適なパラメータの組み合わせを探索するプロセスです。

| 手法 | 説明 | 特徴 |
|------|------|------|
| **Grid Search** | 全ての組み合わせを試す | 確実だが時間がかかる |
| **Random Search** | ランダムに組み合わせを選ぶ | 効率的、広い探索空間向け |
| **Bayesian Optimization** | 過去の結果から次を決める | 効率的、少ない試行回数 |

今回は **RandomizedSearchCV** を使用します（内部でCVを使ってパラメータを評価）。


In [ ]:
# sklearn用にPandas DataFrameへ変換
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

train_pd = train_df.to_pandas()
X_train = train_pd[FEATURE_COLS]
y_train = train_pd[LABEL_COL]

# ハイパーパラメータの探索空間を定義
param_distributions = {
    'max_depth': randint(3, 10),              # 3〜9
    'learning_rate': uniform(0.01, 0.29),     # 0.01〜0.30
    'n_estimators': randint(50, 300),         # 50〜299
    'min_child_weight': randint(1, 10),       # 1〜9
    'subsample': uniform(0.6, 0.4),           # 0.6〜1.0
    'colsample_bytree': uniform(0.6, 0.4),    # 0.6〜1.0
    'scale_pos_weight': randint(1, 5)         # 1〜4（クラス不均衡対策）
}

print("=== 探索するハイパーパラメータ空間 ===")
for param, dist in param_distributions.items():
    print(f"  {param}: {dist}")


In [ ]:
# RandomizedSearchCV の実行
base_model = xgb.XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

random_search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=5,           # 5パターンを試す（時間と精度のバランス）
    scoring='f1',       # F1スコアを最適化（Precision/Recallのバランス）
    cv=3,               # 3-Fold CV（内部でCVを使用）
    verbose=2,
    random_state=42,
    n_jobs=-1           # 並列処理（コンテナランタイムで高速化）
)

print("🔄 ハイパーパラメータチューニング実行中...")
print("   （5パターン × 3-Fold CV = 15回の学習）")
random_search.fit(X_train, y_train)
print("✅ チューニング完了！")


# 最良モデルを取得
best_xgb_model = random_search.best_estimator_

In [ ]:
# 最良パラメータの確認
print("=" * 60)
print("🏆 最良ハイパーパラメータ")
print("=" * 60)
for param, value in random_search.best_params_.items():
    print(f"  {param:<20}: {value}")
print("-" * 60)
print(f"  チューニング時のCV F1: {random_search.best_score_:.4f}")
print("=" * 60)

# 最良モデルを取得
best_xgb_model = random_search.best_estimator_

---
## 3.5 クロスバリデーション（汎化性能の確認）

**ハイパーパラメータチューニングで見つけた最良パラメータ** を使って、
5-Fold クロスバリデーションを実行し、汎化性能を確認します。

In [ ]:
# 最良パラメータで 5-Fold クロスバリデーション
from sklearn.model_selection import cross_val_score

print("🔄 5-Fold クロスバリデーション実行中...")

cv_scores = cross_val_score(best_xgb_model, X_train, y_train, cv=5, scoring='f1')

print("=" * 50)
print("📊 クロスバリデーション結果 (F1)")
print("=" * 50)
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f}")
print("-" * 50)
print(f"  平均:    {cv_scores.mean():.4f}")
print(f"  標準偏差: {cv_scores.std():.4f}")
print("=" * 50)

if cv_scores.std() < 0.05:
    print("✅ 標準偏差が小さく、モデルは安定しています")
else:
    print("⚠️ 標準偏差が大きめ、過学習の可能性")

In [ ]:
# チューニング前後のスコア比較
# チューニング前（デフォルトパラメータ）
default_model = xgb.XGBClassifier(
    max_depth=5, learning_rate=0.1, n_estimators=100,
    scale_pos_weight=2, random_state=42,
    use_label_encoder=False, eval_metric='logloss'
)
default_scores = cross_val_score(default_model, X_train, y_train, cv=3, scoring='f1')

# チューニング後（最良パラメータ）
tuned_scores = cross_val_score(best_xgb_model, X_train, y_train, cv=3, scoring='f1')

print("=" * 60)
print("📊 チューニング前後の比較 (F1)")
print("=" * 60)
print(f"  チューニング前: {default_scores.mean():.4f} (±{default_scores.std():.4f})")
print(f"  チューニング後: {tuned_scores.mean():.4f} (±{tuned_scores.std():.4f})")
print("-" * 60)
improvement = (tuned_scores.mean() - default_scores.mean()) / default_scores.mean() * 100
print(f"  改善率: {improvement:+.2f}%")
print("=" * 60)

---
## 3.6 Feature Importance（特徴量重要度）

モデルの予測にどの特徴量が最も影響しているかを可視化します。

In [ ]:
# Feature Importanceを取得・可視化
import pandas as pd

importance = best_xgb_model.feature_importances_
feature_importance_df = pd.DataFrame({
    'feature': FEATURE_COLS,
    'importance': importance
}).sort_values('importance', ascending=True)

# 可視化
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(feature_importance_df['feature'], feature_importance_df['importance'], color='steelblue')
ax.set_xlabel('Importance (Gain)')
ax.set_title('Feature Importance - Tuned XGBoost')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

# ランキング表示
print("\n=== Feature Importance ランキング ===")
for i, row in feature_importance_df.sort_values('importance', ascending=False).iterrows():
    print(f"  {row['feature']:<25}: {row['importance']:.4f}")

---
## 3.7 SHAP値による解釈

**SHAP** はゲーム理論に基づくモデル解釈手法です。
「なぜこの顧客がチャーンと予測されたのか？」を説明できます。

In [ ]:
# SHAP値の計算
print("🔄 SHAP値を計算中...")
explainer = shap.TreeExplainer(best_xgb_model)

# サンプルデータでSHAP値を計算
X_sample = X_train.sample(n=min(500, len(X_train)), random_state=42)
shap_values = explainer.shap_values(X_sample)

print("✅ SHAP値の計算完了")

In [ ]:
# SHAP Summary Plot
print("=== SHAP Summary Plot ===")
shap.summary_plot(shap_values, X_sample, feature_names=FEATURE_COLS, show=False)
plt.title("SHAP Summary Plot - Feature Impact on Churn")
plt.tight_layout()
plt.show()

In [ ]:
# 個別顧客の予測解釈（Waterfall Plot）
print("=== 個別顧客の予測解釈 ===")

y_pred_sample = best_xgb_model.predict(X_sample)
churn_indices = np.where(y_pred_sample == 1)[0]

if len(churn_indices) > 0:
    idx = churn_indices[0]
    shap.waterfall_plot(shap.Explanation(
        values=shap_values[idx],
        base_values=explainer.expected_value,
        data=X_sample.iloc[idx],
        feature_names=FEATURE_COLS
    ), show=False)
    plt.tight_layout()
    plt.show()
else:
    print("サンプルにチャーン予測顧客がいませんでした")

---
## 3.8 テストデータでの予測

ハイパーパラメータチューニングで見つけた最良モデル（`best_xgb_model`）で、
テストデータを予測します。

In [ ]:
# テストデータで予測（sklearn版モデルを使用）
test_pd = test_df.to_pandas()
X_test = test_pd[FEATURE_COLS]
y_test = test_pd[LABEL_COL]

# 予測実行
y_pred = best_xgb_model.predict(X_test)
y_pred_proba = best_xgb_model.predict_proba(X_test)[:, 1]

# 結果をDataFrameに追加
test_pd["PREDICTED_CHURN"] = y_pred
test_pd["CHURN_PROBABILITY"] = y_pred_proba

print("✅ 予測完了！")
print(f"\nテストデータ件数: {len(test_pd):,}")
print(f"チャーン予測数: {(y_pred == 1).sum():,}")

# 結果の確認
test_pd[[ID_COL, LABEL_COL, "PREDICTED_CHURN", "CHURN_PROBABILITY"]].head(10)

---
## 3.9 モデルの評価

In [ ]:
# 評価メトリクスの計算
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_true = test_pd[LABEL_COL]
y_pred = test_pd["PREDICTED_CHURN"]

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("=" * 50)
print("📊 モデル評価結果（テストデータ）")
print("=" * 50)
print(f"  Accuracy  : {accuracy:.4f}")
print(f"  Precision : {precision:.4f}")
print(f"  Recall    : {recall:.4f}")
print(f"  F1 Score  : {f1:.4f}")
print("=" * 50)

In [ ]:
# 混同行列
cm = confusion_matrix(y_true, y_pred)

print("=== 混同行列 ===")
print(f"              予測:Active  予測:Churn")
print(f"実際:Active    {cm[0][0]:>8}    {cm[0][1]:>8}")
print(f"実際:Churn     {cm[1][0]:>8}    {cm[1][1]:>8}")

---
## 3.10 予測結果の保存

In [ ]:
# 予測結果をテーブルに保存
predictions_df = session.create_dataframe(test_pd)
predictions_df.write.save_as_table(
    "MLOPS_HOL_DB.FEATURE_STORE.MODEL_PREDICTIONS_V1",
    mode="overwrite"
)

print("✅ 予測結果を保存しました")
print(f"   テーブル: MODEL_PREDICTIONS_V1")
print(f"   レコード数: {len(test_pd):,}")

In [ ]:
# モデルを変数に保存（次のセクションで使用）
trained_model = best_xgb_model
baseline_metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1_score": f1
}

print("✅ モデルを変数 'trained_model' に保存")
print(f"   最良パラメータ: {random_search.best_params_}")
print("   次のセクションでExperiment Trackingに登録します")

---
## ✅ Section 3 完了！

### 学んだこと
- **ハイパーパラメータチューニング** (RandomizedSearchCV)
- **クロスバリデーション** で汎化性能確認
- **Feature Importance** で重要特徴量の特定
- **SHAP値** でモデルの解釈可能性

### 作成したもの
- チューニング済みチャーン予測モデル
- 予測結果テーブル: `MODEL_PREDICTIONS_V1`

### 次のステップ
**Section 4: Experiment Tracking** では、複数の実験を管理します。

➡️ `04_EXPERIMENT_TRACKING` Notebookに進んでください